In [ ]:
# Instalação das dependências
!pip install -U langgraph requests langchain-openai openai python-dotenv langchain langchain-community

In [ ]:
import os
import re
import json
import requests
from typing import Annotated, List
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools import tool
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    openai_api_base="http://plataforma.sobdemanda.mandu.piaui.pro",
    openai_api_key="sk-60gBh1n5AVAsgJ2HZKq4HA",
    model="Qwen3-30B-A3B",
    temperature=0.8
)

# --- Estrutura do estado ---
class IntencaoUsuario(TypedDict):
    intencao: str
    cod_processo: str

class ChatState(TypedDict):
    messages: Annotated[List, add_messages]
    intencao_usuario: IntencaoUsuario
    is_valid: bool

# --- Nós do LangGraph ---
def qwen_chat_intencao(state: ChatState) -> ChatState:
    try:
        user_input = state["messages"][-1]

        assistent_prompt = (
            'Você é um assistente jurídico. Analise a mensagem abaixo e retorne apenas um JSON no formato: '
            '{"intencao": str, "cod_processo": str}. '
            'Valores válidos para "intencao": "resumo" ou "andamento". '
            '"cod_processo" deve seguir o formato 99999.999999/9999-99.'
        )

        response = llm.invoke([
            {"role": "system", "content": assistent_prompt},
            {"role": "user", "content": user_input.content}
        ])

        print("Conteúdo recebido do modelo:\n", response.content)

        try:
            resposta_json = json.loads(response.content.strip("```json").strip("```").strip())
        except json.JSONDecodeError:
            erro_msg = (
                "O modelo não retornou JSON válido.\n"
                f"Conteúdo recebido:\n{response.content}"
            )
            state["messages"].append(AIMessage(content=erro_msg))
            state["intencao_usuario"] = {"intencao": "", "cod_processo": ""}
            return state

        state["intencao_usuario"] = resposta_json
        intencao = resposta_json.get("intencao", "")
        state["messages"].append(AIMessage(content=f"Intenção detectada: '{intencao}'"))

        return state

    except Exception as e:
        return {"messages": [AIMessage(content=f"Erro inesperado: {str(e)}")]}

def verificar_numero_processo(state: ChatState) -> ChatState:
    processo = state["intencao_usuario"].get("cod_processo", "")
    padrao = r"([0-9]{5,})\.?([0-9]{6,})\/?([0-9]{4,})-?([0-9]{2,})"
    state["is_valid"] = bool(re.match(padrao, processo))
    return state

def executar_intencao_processo(state: ChatState) -> ChatState:
    intencao = state["intencao_usuario"].get("intencao", "")
    processo = state["intencao_usuario"].get("cod_processo", "")
    valido = state.get("is_valid", False)

    if not valido:
        resposta = f"Número de processo inválido: {processo}."
        state["messages"].append(AIMessage(content=resposta))
        return state

    try:
        processo_formatado = processo.replace(".", "").replace("/", "").replace("-", "")

        if intencao == "resumo":
            response_resumo = requests.get(f"http://127.0.0.1:8000/resumo/{processo_formatado}")
            response_resumo.raise_for_status()
            dados_processo = response_resumo.json()
            resposta = f"Resumo do processo {processo}:\n\n{dados_processo.get('resumo', 'Resumo não encontrado.')}"
        elif intencao == "andamento":
            response_andamento = requests.get(f"http://127.0.0.1:8000/andamento/{processo_formatado}")
            response_andamento.raise_for_status()
            dados_processo = response_andamento.json()
            resposta = f"Andamento do processo {processo}:\n\n{dados_processo.get('andamento', 'Andamento não encontrado.')}"
        else:
            resposta = f"Intenção '{intencao}' não reconhecida ou não suportada."

    except Exception as e:
        resposta = f"Erro ao consultar API local: {str(e)}"

    state["messages"].append(AIMessage(content=resposta))
    return state



workflow = StateGraph(ChatState)
workflow.add_node("qwen_chat_intencao", qwen_chat_intencao)
workflow.add_node("verificar_numero_processo", verificar_numero_processo)
workflow.add_node("executar_intencao_processo", executar_intencao_processo)
workflow.add_edge(START, "qwen_chat_intencao")
workflow.add_edge("qwen_chat_intencao", "verificar_numero_processo")
workflow.add_edge("verificar_numero_processo", "executar_intencao_processo")
workflow.add_edge("executar_intencao_processo", END)

chatbot = workflow.compile()

entrada_usuario = "me fale sobre o resumo do processo 00002.003059/2024-15"
initial_state = {
    "messages": [HumanMessage(content=entrada_usuario)],
    "intencao_usuario": {"intencao": "", "cod_processo": ""},
    "is_valid": False
}

resultado = chatbot.invoke(initial_state)

for m in resultado["messages"]:
    print(f"{m.type.upper()}: {m.content}")

In [ ]:
import re
import json
import requests
from typing import Annotated, List
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_core.messages import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    openai_api_base="http://plataforma.sobdemanda.mandu.piaui.pro",
    openai_api_key="sk-60gBh1n5AVAsgJ2HZKq4HA",
    model="Qwen3-30B-A3B",
    temperature=0.8
)

class IntencaoUsuario(TypedDict):
    intencao: str
    cod_processo: str

class ChatState(TypedDict):
    messages: Annotated[List, add_messages]
    intencao_usuario: IntencaoUsuario
    is_valid: bool

def responder_com_ia(texto_api: str, processo: str, intencao: str) -> str:
    prompt = (
        f"Com base nas informações abaixo, responda ao usuário sobre o processo {processo}.\n\n"
        f"{texto_api}"
    )

    resposta = llm.invoke([
        {"role": "user", "content": prompt}
    ])

    return resposta.content

def qwen_chat_intencao(state: ChatState) -> ChatState:
    try:
        user_input = state["messages"][-1]

        assistent_prompt = (
            'Analise a mensagem abaixo e retorne apenas um JSON no formato: '
            '{"intencao": str, "cod_processo": str}. '
            'Valores válidos para "intencao": "resumo" ou "andamento". '
            '"cod_processo" deve seguir o formato 99999.999999/9999-99.'
        )

        response = llm.invoke([
            {"role": "system", "content": assistent_prompt},
            {"role": "user", "content": user_input.content}
        ])

        print("Conteúdo recebido do modelo:\n", response.content)

        try:
            resposta_json = json.loads(response.content.strip("```json").strip("```").strip())
        except json.JSONDecodeError:
            erro_msg = (
                "O modelo não retornou JSON válido.\n"
                f"Conteúdo recebido:\n{response.content}"
            )
            state["messages"].append(AIMessage(content=erro_msg))
            state["intencao_usuario"] = {"intencao": "", "cod_processo": ""}
            return state

        state["intencao_usuario"] = resposta_json
        intencao = resposta_json.get("intencao", "")
        state["messages"].append(AIMessage(content=f"Intenção detectada: '{intencao}'"))

        return state

    except Exception as e:
        return {"messages": [AIMessage(content=f"Erro inesperado: {str(e)}")]}

def verificar_numero_processo(state: ChatState) -> ChatState:
    processo = state["intencao_usuario"].get("cod_processo", "")
    padrao = r"([0-9]{5})\.([0-9]{6})\/([0-9]{4})-([0-9]{2})"
    state["is_valid"] = bool(re.match(padrao, processo))
    return state

def executar_intencao_processo(state: ChatState) -> ChatState:
    intencao = state["intencao_usuario"].get("intencao", "")
    processo = state["intencao_usuario"].get("cod_processo", "")
    valido = state.get("is_valid", False)

    if not valido:
        resposta = f"Número de processo inválido: {processo}."
        state["messages"].append(AIMessage(content=resposta))
        return state

    try:
        processo_formatado = processo.replace(".", "").replace("/", "").replace("-", "")

        if intencao == "resumo":
            response = requests.get(f"http://127.0.0.1:8000/resumo/{processo_formatado}")
        elif intencao == "andamento":
            response = requests.get(f"http://127.0.0.1:8000/andamento/{processo_formatado}")
        else:
            resposta = f"Intenção '{intencao}' não reconhecida ou não suportada."
            state["messages"].append(AIMessage(content=resposta))
            return state

        response.raise_for_status()
        texto_api = response.text.strip()

        resposta_texto = responder_com_ia(texto_api, processo, intencao)
        state["messages"].append(AIMessage(content=resposta_texto))

    except Exception as e:
        resposta = f"Erro ao consultar API local: {str(e)}"
        state["messages"].append(AIMessage(content=resposta))

    return state

workflow = StateGraph(ChatState)
workflow.add_node("qwen_chat_intencao", qwen_chat_intencao)
workflow.add_node("verificar_numero_processo", verificar_numero_processo)
workflow.add_node("executar_intencao_processo", executar_intencao_processo)

workflow.add_edge(START, "qwen_chat_intencao")
workflow.add_edge("qwen_chat_intencao", "verificar_numero_processo")
workflow.add_edge("verificar_numero_processo", "executar_intencao_processo")
workflow.add_edge("executar_intencao_processo", END)

chatbot = workflow.compile()

entrada_usuario = "me fale sobre o resumo do processo 00002.003059/2024-15"

initial_state = {
    "messages": [HumanMessage(content=entrada_usuario)],
    "intencao_usuario": {"intencao": "", "cod_processo": ""},
    "is_valid": False
}

resultado = chatbot.invoke(initial_state)

for m in resultado["messages"]:
    print(f"{m.type.upper()}: {m.content}")


Conteúdo recebido do modelo:
 

{"intencao": "resumo", "cod_processo": "00002.003059/2024-15"}
HUMAN: me fale sobre o resumo do processo 00002.003059/2024-15
AI: Intenção detectada: 'resumo'
AI: 

O processo **00002.003059/2024-15** refere-se a um **memorando oficial** emitido pelo **Núcleo Estratégico de Tecnologia e Governo Digital (NTGD)** da **Secretaria de Administração do Estado do Piauí (SEAD-PI)**. Abaixo estão os principais detalhes:

### **Informações Principais**:
- **Tipo de Documento**: Memorando (SEAD_MEMORANDO).  
- **Número do Processo**: 00002.003059/2024-15.  
- **Data de Elaboração**: 09 de abril de 2024.  
- **Unidade Solicitante**: Núcleo Estratégico de Tecnologia e Governo Digital (NTGD) – SEAD-PI.  
- **Assunto**: Solicitação de autorização para contratação de **estagiário do curso de Engenharia de Software**.  
- **Destinatário**: Superintendência de Gestão de Pessoas (SGP) – SEAD-PI.  

### **Detalhes da Solicitação**:
- **Objetivo**: Inclusão de um estagiário 